In [1]:
import os
import tarfile
import urllib
import re
import bz2
import pandas as pd

In [2]:
DOWNLOAD_ROOT = "https://spamassassin.apache.org"
SPAM_PATH = os.path.join("datasets", "spam")
SPAM_URL = DOWNLOAD_ROOT + "/old/publiccorpus/"

def fetch_spam_data(spam_url=SPAM_URL, spam_path=SPAM_PATH):
    os.makedirs(spam_path, exist_ok=True)
    urlpath = urllib.request.urlopen('https://spamassassin.apache.org/old/publiccorpus/')
    string = urlpath.read().decode('utf-8')
    pattern = re.compile('href=".*\.bz2"')
    filelist = [f[6:-1] for f in pattern.findall(string)]
    for f in filelist:
        bz2path = os.path.join(spam_path, f)
        urllib.request.urlretrieve(spam_url+f, bz2path)
        with bz2.BZ2File(bz2path) as bz2_file:
            data = bz2_file.read()
        tarpath = bz2path[:-4]
        open(tarpath, 'wb').write(data)
        with tarfile.open(tarpath) as tar_file:
            tar_file.extractall(path=spam_path)
        os.remove(bz2path)
        os.remove(tarpath)

In [3]:
# fetch_spam_data()

In [82]:
contents = []
cats = []

for cat in os.listdir(SPAM_PATH):
    cat_path = os.path.join(SPAM_PATH, cat)
    cat_files = os.listdir(cat_path)
    cat_contents = [open(os.path.join(cat_path, f), encoding='ansi').read() for f in cat_files if f != "cmds"]
    contents += cat_contents
    cats += [cat] * len(cat_contents)
    
emails = pd.DataFrame(data={'Content': contents, 'Cat': cats})

In [83]:
emails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9349 entries, 0 to 9348
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Content  9349 non-null   object
 1   Cat      9349 non-null   object
dtypes: object(2)
memory usage: 146.2+ KB


In [84]:
emails['Cat'].value_counts()

easy_ham      5051
easy_ham_2    1400
spam_2        1397
spam          1001
hard_ham       500
Name: Cat, dtype: int64

In [85]:
emails["Cat"] = emails.apply(lambda row: row["Cat"][:-2] if (row["Cat"][-2] == '_' and row["Cat"][-1].isdigit()) else row["Cat"], axis=1)

In [86]:
emails['Cat'].value_counts()/len(emails)

easy_ham    0.690020
spam        0.256498
hard_ham    0.053482
Name: Cat, dtype: float64

In [87]:
emails['Label'] = emails['Cat'].str.contains('spam').astype(int)

In [88]:
emails

,Content,Cat,Label
0,From exmh-workers-admin@redhat.com Thu Aug 22...,easy_ham,0
1,From Steve_Burt@cursor-system.com Thu Aug 22 ...,easy_ham,0
2,From timc@2ubh.com Thu Aug 22 13:52:59 2002\n...,easy_ham,0
3,From irregulars-admin@tb.tf Thu Aug 22 14:23:...,easy_ham,0
4,From Stewart.Smith@ee.ed.ac.uk Thu Aug 22 14:...,easy_ham,0
...,...,...,...
9344,From Professional_Career_Development_Institute...,spam,1
9345,From tba@insiq.us Wed Dec 4 11:46:34 2002\nR...,spam,1
9346,Return-Path: <raye@yahoo.lv>\nReceived: from u...,spam,1
9347,From cweqx@dialix.oz.au Tue Aug 6 11:03:54 2...,spam,1


In [89]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(emails, emails["Cat"]):
    strat_train_set = emails.loc[train_index]
    strat_test_set = emails.loc[test_index]

In [90]:
strat_train_set['Cat'].value_counts()/len(strat_train_set)

easy_ham    0.690066
spam        0.256451
hard_ham    0.053483
Name: Cat, dtype: float64

In [91]:
strat_test_set['Cat'].value_counts()/len(strat_test_set)

easy_ham    0.689840
spam        0.256684
hard_ham    0.053476
Name: Cat, dtype: float64

In [93]:
emails

,Content,Cat,Label
0,From exmh-workers-admin@redhat.com Thu Aug 22...,easy_ham,0
1,From Steve_Burt@cursor-system.com Thu Aug 22 ...,easy_ham,0
2,From timc@2ubh.com Thu Aug 22 13:52:59 2002\n...,easy_ham,0
3,From irregulars-admin@tb.tf Thu Aug 22 14:23:...,easy_ham,0
4,From Stewart.Smith@ee.ed.ac.uk Thu Aug 22 14:...,easy_ham,0
...,...,...,...
9344,From Professional_Career_Development_Institute...,spam,1
9345,From tba@insiq.us Wed Dec 4 11:46:34 2002\nR...,spam,1
9346,Return-Path: <raye@yahoo.lv>\nReceived: from u...,spam,1
9347,From cweqx@dialix.oz.au Tue Aug 6 11:03:54 2...,spam,1


In [101]:
from sklearn.base import BaseEstimator, TransformerMixin
import email
from email_reply_parser import EmailReplyParser

# column index
content_ix = 0

class MyEmailParser(BaseEstimator, TransformerMixin):
    def __init__(self, strip_headers=True): # no *args or **kargs
        self.strip_headers = strip_headers
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        if self.strip_headers:
            new_contents = []
            for email_content in X[:, content_ix]:
                new = ''
                message = email.message_from_string(email_content)
                if message.is_multipart():
                    for payload in message.get_payload():
                        new += payload.get_payload()
                else:
                    new = message.get_payload()
                new_contents += [new]
            X[:, content_ix] = np.array(new_contents)

        return X

# attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
# housing_extra_attribs = attr_adder.transform(housing.values)

NameError: name 'CombinedAttributesAdder' is not defined

In [136]:
new_contents = []
for email_content in emails[emails.Content.str.contains('text/html')].values[0:3, 0]:
    new = ''
    message = email.message_from_string(email_content)
    if message.is_multipart():
        for payload in message.get_payload():
            new += payload.get_payload()
    else:
        new = message.get_payload()
    new_contents += [new]
stripado = np.array(new_contents)

In [138]:
print(stripado[0])

I'm using Simple DNS from JHSoft.  We support only a few web sites and =
I'd like to swap secondary services with someone in a similar position.

We have a static IP, DSL line and a 24/7 set of web, SQL, mail and now a =
DNS server.  As I said, we are hosting about 10 web sites, web and DNS =
traffic is almost nothing.  Everything is on lightly loaded APC battery =
backups so we are very seldom down.

I'd like to swap with someone also using Simple DNS to take advantage of =
the trusted zone file transfer option.



Bob Musser
Database Services, Inc.
Makers of:
   Process Server's Toolbox
   Courier Service Toolbox
BobM@dbsinfo.com
www.dbsinfo.com
106 Longhorn Road
Winter Park FL 32792
(407) 679-1539


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">
<HTML><HEAD>
<META http-equiv=3DContent-Type content=3D"text/html; =
charset=3Dwindows-1252">
<META content=3D"MSHTML 6.00.2716.2200" name=3DGENERATOR>
<STYLE></STYLE>
</HEAD>
<BODY bgColor=3D#ffffff>
<DIV><FONT size=3D2>I'm 

In [135]:
print(emails[emails.Content.str.contains('text/html')].values[0, 0])

From DNS-swap@lists.ironclad.net.au  Mon Sep  2 12:28:53 2002
Return-Path: <DNS-swap@lists.ironclad.net.au>
Delivered-To: zzzz@localhost.netnoteinc.com
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id 40C6C43F9B
	for <zzzz@localhost>; Mon,  2 Sep 2002 07:28:52 -0400 (EDT)
Received: from phobos [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for zzzz@localhost (single-drop); Mon, 02 Sep 2002 12:28:52 +0100 (IST)
Received: from lists.ironclad.net.au ([203.30.247.12]) by
    dogma.slashnull.org (8.11.6/8.11.6) with SMTP id g7UFQvZ30600 for
    <zzzz@spamassassin.taint.org>; Fri, 30 Aug 2002 16:27:01 +0100
Received: from dbsinfo.com by lists.ironclad.net.au with SMTP;
    Sat, 31 Aug 2002 01:21:00 +1000
Message-Id: <00c401c25039$7b055460$976fa8c0@cfl.rr.com>
From: "Bob Musser" <BobM@dbsinfo.com>
To: <DNS-swap@lists.ironclad.net.au>
Subject: Tiny DNS Swap
Date: Fri, 30 Aug 2002 11:25:31 -0400
MIME-Version: 1.0
Content-Type: multi

URL Parser -> detecta http e https URL_TAG -> ver beautifulsoup

HTML -> tira todas as tags

$, euro, libra, yens etc pra MONEY_TAG

numeros para NUMBER_TAG

tokenizar

stemming

tirar stopwords

selecionar só as palavras mais frequentes

In [ ]:
class MyHTMLParser(BaseEstimator, TransformerMixin):
    def __init__(self, strip_headers=True): # no *args or **kargs
        self.strip_headers = strip_headers
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        if self.strip_headers:
            new_contents = []
            for email_content in X[:, content_ix]:
                new = ''
                message = email.message_from_string(email_content)
                if message.is_multipart():
                    for payload in message.get_payload():
                        new += payload.get_payload()
                else:
                    new = message.get_payload()
                new_contents += [new]
            X[:, content_ix] = np.array(new_contents)

        return X

# Pipeline

contar a ocorrencia de cada palavra (binario ou frequencia?)

selecionar mais frequentes

tirar stopwords

pontuação provavelmente contar (!, ?)

tirar header do email (separar texto na primeira quebra de linha dupla (acho)) ou nao tirar?? pode avaliar assunto e quem enviou

substituir urls por URL, numeros por NUMBER, $ por MONEY talvez

stemming

tratar html </>

replies? \t |